In [22]:

import urllib.request
import pandas as pd
import requests
import io
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot
from statsmodels.discrete.discrete_model import MNLogit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
hsb2 = pd.read_csv("C:\\Users\\Admin\\Desktop\\Vaishnavi\\Auto\\DriverData1.csv")

hsb2["Gap"] = hsb2["Gap"].astype('int64')
hsb2["RelativeSpeed"] = hsb2["RelativeSpeed"].astype('int64')
hsb2["TW"] = hsb2["TW"].astype('category')
hsb2['Choice']=hsb2['Choice']-1

hsb3 = hsb2
hsb3.head()

,ID,SubVehID,Choice,Gap,RelativeSpeed,TW
0,1,9,1,12,0,0
1,2,13,0,16,2,1
2,3,13,2,15,1,1
3,4,13,0,15,0,1
4,5,36,1,12,0,0


In [23]:
y = hsb3['Choice']
X = hsb3.drop(['ID','SubVehID','Choice'],axis=1)

In [26]:
import numpy as np
from scipy.optimize import minimize

# Define the softmax function
def softmax(z):
    return np.exp(z) / np.sum(np.exp(z), axis=1, keepdims=True)

# Define the log-likelihood function
def log_likelihood(params, X, y, num_classes):
    num_samples, num_features = X.shape
    beta = params.reshape(num_classes, num_features + 1)
    X_intercept = np.hstack([np.ones((num_samples, 1)), X])
    logits = np.dot(X_intercept, beta.T)
    probabilities = softmax(logits)
    log_likelihood = np.sum(np.log(probabilities[np.arange(num_samples), y]))
    return -log_likelihood

# Define a function to fit the model using L-BFGS-B method
def fit_multinomial_logistic_regression(X, y, num_classes, max_iter=1000000):
    num_samples, num_features = X.shape
    initial_params = np.zeros((num_classes, num_features + 1)).ravel()

    # Optimize the log-likelihood function using L-BFGS-B method
    result = minimize(log_likelihood, initial_params, args=(X, y, num_classes),
                      method='L-BFGS-B', jac=None,
                      options={'maxiter': max_iter, 'disp': True})

    beta = result.x.reshape(num_classes, num_features + 1)
    return beta

# Example usage:
# X and y should be defined with appropriate data for your multinomial logistic regression problem
# num_classes should be set to the number of classes in your problem

beta = fit_multinomial_logistic_regression(X, y, num_classes)
print("Intercept and Coefficients:")
print(beta)


Intercept and Coefficients:
[[ 2.14666262 -0.12960308 -0.20794964  0.15761092]
 [-0.45839896 -0.0203722  -0.07868068  0.05502989]
 [-1.68828305  0.14947839  0.28660041 -0.21266022]]


In [25]:

import pandas as pd
import statsmodels.api as sm
from scipy import stats
from sklearn.metrics import log_loss
# Create a multinomial logistic regression model


model =LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', 
                           C=1.0, max_iter=1000000)

# Fit the model to the data
model.fit(X, y)
print("Intercept:",model.intercept_)
print("Coefficients:",model.coef_)
# Get the log-likelihood of the fitted model
y_prob = model.predict_proba(X)
LL_model = -log_loss(y, y_prob, normalize=False)

# Fit the null model (only an intercept)
null_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', 
                                C=1.0, max_iter=1000000, fit_intercept=True)
X_null = np.ones((X.shape[0], 1))  # Only intercept
null_model.fit(X_null, y)

# Get the log-likelihood of the null model
y_null_prob = null_model.predict_proba(X_null)
LL_null = -log_loss(y, y_null_prob, normalize=False)


chi_square_stat = 2 * (LL_model - LL_null)
df_model = X.shape[1] * (len(np.unique(y)) - 1) 
df_null = len(np.unique(y)) - 1  # Number of intercepts in the null model
df_diff = df_model - df_null
p_value = stats.chi2.sf(chi_square_stat, df_diff)
rho_squared = 1 - (LL_model / LL_null)
print("Rho-squared:", rho_squared)
print("Chi-square statistic:", chi_square_stat)
print("p-value:", p_value)


Intercept: [ 2.14744897 -0.45742293 -1.69002604]
Coefficients: [[-0.12946688 -0.20774494  0.15692087]
 [-0.02023765 -0.07849998  0.0538605 ]
 [ 0.14970453  0.28624493 -0.21078137]]
Rho-squared: 0.12235023609532403
Chi-square statistic: 577.9456458256163
p-value: 9.184486612176098e-124


In [134]:

c = []
# Initialize an empty list to store predicted probabilities
for i in range(len(hsb2.ID)):
    row = X.iloc[i:i+1, :]  # Select a single row from the DataFrame
    # Predict a multinomial probability distribution
    yhat = model.predict(row)
    # Summarize the predicted probabilities and append to the list
    c.append(yhat[0])
    

hsb2['Predicted'] = c

In [135]:
# Function to predict probabilities
def predict_probabilities(X, beta):
    X_intercept = np.hstack([np.ones((X.shape[0], 1)), X])
    logits = np.dot(X_intercept, beta.T)
    probabilities = softmax(logits)
    return probabilities

# Predict probabilities and class labels
c = []
y_pred = []

for i in range(len(hsb2)):
    row = X.iloc[i:i+1, :]  # Select a single row from the DataFrame
    # Predict a multinomial probability distribution
    phat = predict_probabilities(row, beta)
    # Summarize the predicted probabilities and append to the list
    c.append(phat[0])
    # Determine the predicted class label (the one with the highest probability)
    y_pred.append(np.argmax(phat))
hsb2['Predicted_1'] = y_pred

In [144]:
# Fitting the MNLogit model
X1 = sm.add_constant(X)

model = sm.MNLogit(y, X1)
result = model.fit()

# Display the summary of the model
print(result.summary())


Optimization terminated successfully.
         Current function value: 0.863698
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:                 Choice   No. Observations:                 2400
Model:                        MNLogit   Df Residuals:                     2392
Method:                           MLE   Df Model:                            6
Date:                Thu, 13 Jun 2024   Pseudo R-squ.:                  0.1224
Time:                        01:54:31   Log-Likelihood:                -2072.9
converged:                       True   LL-Null:                       -2361.8
Covariance Type:            nonrobust   LLR p-value:                1.331e-121
     Choice=1       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -2.6055      0.604     -4.311      0.000      -3.790      -1.421
Gap               0.

In [145]:
# Predict probabilities
pred_probs = result.predict(X1)

# Predict the class with the highest probability
y_pred = np.argmax(pred_probs.values, axis=1)

# Add predictions to the dataframe
hsb2['Predicted_Class'] = y_pred



In [146]:
hsb2.to_csv("Results.csv", index=False)